In [350]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from mpl_toolkits.mplot3d import Axes3D
from pandas.tools.plotting import scatter_matrix
from pylab import rcParams
import seaborn as sb
import matplotlib.pyplot as plt
import pickle
import warnings
from sklearn.externals import joblib
warnings.filterwarnings('ignore')

In [351]:
    input_folder_path= r"../Data"
    #filename1 = "/WalmartDatasset.txt"
    filename1 = "/nov01-30days.txt"
    filename2 = "/nov21-30days.txt"
    
    header_names=["term","product_id", "language","product_impressions","Product_clicks","cart_adds","cart_start","checkout","order"]
    df_dly_hdp = pd.read_csv(input_folder_path+filename1, sep='\t', names = header_names)
    df_dly_hdp1 = pd.read_csv(input_folder_path+filename2, sep='\t', names = header_names)
    frames = [df_dly_hdp,df_dly_hdp1]
    result = pd.concat(frames,ignore_index=True)

In [352]:
    # filter out French language records
    result = (result[result['language'] == 'English'])

    joined_df_dly = (
    result[['term', 'product_id', 'product_impressions', 'Product_clicks', 'cart_adds', 'checkout', 'order']])

In [353]:
# print(df_dly_hdp)


joined_df_dly['searchTerm'] = joined_df_dly['term']
joined_df_dly['ATR'] = joined_df_dly['cart_adds'] / joined_df_dly['product_impressions']
joined_df_dly['CTR'] = joined_df_dly['Product_clicks'] / joined_df_dly['product_impressions']
joined_df_dly['conv'] = joined_df_dly['order'] / joined_df_dly['product_impressions']
joined_df_dly['ATRmCTR'] = joined_df_dly['ATR'] * joined_df_dly['CTR']

columns_all = ['searchTerm', 'product_id', 'product_impressions', 'Product_clicks', 'cart_adds', 'order', 'CTR', 'ATR',
               'conv','ATRmCTR']

In [354]:
# columns for input

joined_df_dly = joined_df_dly[list(columns_all)].reset_index()

print(joined_df_dly.info())

###########################################3

# print(joined_df_dly.tail(50))

# Concat null results and positive results - Final step

# columns for input

final_merged_dataset = joined_df_dly[list(columns_all)]
final_merged_dataset = final_merged_dataset.replace(np.nan, 0)
final_merged_dataset = final_merged_dataset.drop_duplicates()

final_merged_dataset.to_csv(r"/Users/soumyamohapatra/CMPE-256/Data/final_merged_dataset_mthly2_test.csv", sep=',',
                            index=False)

# result = df_dly_null.append(df_dly_pos, ignore_index=True)
print(final_merged_dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3045860 entries, 0 to 3045859
Data columns (total 11 columns):
index                  int64
searchTerm             object
product_id             object
product_impressions    int64
Product_clicks         int64
cart_adds              int64
order                  int64
CTR                    float64
ATR                    float64
conv                   float64
ATRmCTR                float64
dtypes: float64(4), int64(5), object(2)
memory usage: 255.6+ MB
None
       searchTerm     product_id  product_impressions  Product_clicks  \
0    men t shirts  6000196965847                    7               1   
1     dvi to hdmi  6000066423242                   65              11   
2  rubix cube 4x4  6000196692502                    3               1   
3         Day bed  6000195505799                   10               1   
4           razor  6000195571001                    9               1   

   cart_adds  order       CTR       ATR  conv   AT

In [355]:
#pd.read_csv('final_merged_dataset_mthly2.csv')
df2 = pd.read_csv('../Data/final_merged_dataset_mthly2_test.csv', encoding='latin-1')


In [356]:
##filter value which are not zero , zero values will not aid prediction
df3 = df2[((df2['ATR'] >0)&(df2['CTR'] >0)&(df2['conv'] >0))]
#df3 = df2[df2['conv'] >0]

In [357]:
row_counts = {}
for i in df3['searchTerm']:
    if i in row_counts:
        row_counts[i]+=1
    else:
        row_counts[i]=1

In [358]:
len(row_counts)

51364

In [359]:
highfreq_searchStrings = []
for j in row_counts:
    if row_counts[j] > 50:
        highfreq_searchStrings.append(j)

In [360]:
highfreq_searchStrings = [x for x in highfreq_searchStrings if not any(c.isdigit() for c in x)]

In [361]:
print (highfreq_searchStrings[2])

cat food


In [362]:
##data cleaning as few outliers were present and it was difficult to visualize data relationship , very fat tail
##lot of value in the lower end 
def removeoutliers(highfreq_searchStrings= None):
    if highfreq_searchStrings == None:
        dfsearch = df3
    else:
        dfsearch = df3[(df3['searchTerm'] == highfreq_searchStrings)]
    ATRstd,CTRstd = dfsearch[['ATR','CTR']].std()
    ATRmean,CTRmean = dfsearch[['ATR','CTR']].mean()
    dfsearchWoOutliers = dfsearch[((dfsearch['ATR']<=(ATRmean+2*ATRstd))& (dfsearch['ATR']>=(ATRmean-1*ATRstd)))]
    dfsearchWoOutliers = dfsearchWoOutliers[((dfsearch['CTR']<=(CTRmean+2*CTRstd))& (dfsearch['CTR']>=(CTRmean-1*CTRstd)))]
    return dfsearchWoOutliers

In [363]:
def pairplotgraphs(dfsearchWoOutliers):
    rcParams['figure.figsize'] = 5,5
    sb.set_style('whitegrid')
    #plt.hist(dfsearch['CTR'])
    #plt.plot()
    sb.pairplot(dfsearchWoOutliers,vars=['CTR','ATR','ATRmCTR','conv'], y_vars=['conv'])
    dfsearchWoOutliers.plot(kind='scatter',x='ATR',y='conv',c=['darkgray'],s=150)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(dfsearchWoOutliers['CTR'], dfsearchWoOutliers['ATR'], dfsearchWoOutliers['conv'], c='r', marker='o')
    ax.set_xlabel('CTR')
    ax.set_ylabel('ATR')
    ax.set_zlabel('CONV')
    plt.show()


In [364]:


#X_train,X_test,y_train,y_test = train_test_split(dfsearchWoOutliers[['CTR']],dfsearchWoOutliers[['conv']],test_size=0.40)

In [365]:
abc=[]
def Regressors(dfsearchWoOutliers):
    mean_scores =[]
    reg0 = LinearRegression()
    reg0.fit(dfsearchWoOutliers[['CTR']],dfsearchWoOutliers[['conv']])
    scores0 = cross_val_score(reg0, dfsearchWoOutliers[['CTR']], dfsearchWoOutliers[['conv']])
    #print(reg0.coef_)
    #print(reg0.intercept_)
    #print ("with CTR",scores0.mean())
    mean_scores.append(scores0.mean())
    abc.append(reg0)
    
    
    reg1 = LinearRegression()
    reg1.fit(dfsearchWoOutliers[['ATR']],dfsearchWoOutliers[['conv']])
    scores1 = cross_val_score(reg1, dfsearchWoOutliers[['ATR']], dfsearchWoOutliers[['conv']])
    #print (reg1.coef_)
    #print (reg1.intercept_)
    #print ("with ATR",scores1.mean())
    mean_scores.append(scores1.mean())
    
    reg2 = LinearRegression()
    reg2.fit(dfsearchWoOutliers[['CTR','ATR']],dfsearchWoOutliers[['conv']])
    scores2 = cross_val_score(reg2, dfsearchWoOutliers[['CTR','ATR']], dfsearchWoOutliers[['conv']])
    #results=reg2.predict(df2[['CTR','ATR']])
    #print (results)
    #print (reg2.coef_)
    #print (reg2.intercept_)
    #print ("with ATRandCTR",scores2.mean())
    mean_scores.append(scores2.mean())
    
    reg3 = LinearRegression()
    reg3.fit(dfsearchWoOutliers[['CTR','ATR','ATRmCTR']],dfsearchWoOutliers[['conv']])
    scores3 = cross_val_score(reg3, dfsearchWoOutliers[['CTR','ATR','ATRmCTR']], dfsearchWoOutliers[['conv']])
    #print (reg4.coef_)
    #print (reg4.intercept_)
    #print ("with ATR,CTRandInteraction",scores3.mean())
    mean_scores.append(scores3.mean())
    
    poly2 = PolynomialFeatures(degree=2)
    X2_Poly = poly2.fit_transform(dfsearchWoOutliers[['CTR','ATR']])
    reg4 = LinearRegression()
    reg4.fit(X2_Poly,dfsearchWoOutliers[['conv']])
    scores4 = cross_val_score(reg4, X2_Poly, dfsearchWoOutliers[['conv']])
    #print (reg5.coef_)
    #print (reg5.intercept_)
    #print ("with ATR,CTR2ndDegree",scores4.mean())
    mean_scores.append(scores4.mean())
    
    ##To validate if scaling is changing model parameters
    scaler = MinMaxScaler()
    scaler.fit(X2_Poly)
    scaler.transform(X2_Poly)
    reg41 = LinearRegression()
    reg41.fit(X2_Poly,dfsearchWoOutliers[['conv']])
    scores7 = cross_val_score(reg41, X2_Poly, dfsearchWoOutliers[['conv']])
    #print (reg41.coef_)
    #print (reg41.intercept_)
    scores41 = cross_val_score(reg41, X2_Poly, dfsearchWoOutliers[['conv']])
    #print ("with Scaled ATR,CTR2ndDegree",scores41.mean())
    
    
    poly3 = PolynomialFeatures(degree=3)
    X3_Poly = poly3.fit_transform(dfsearchWoOutliers[['CTR','ATR']])
    reg5 = LinearRegression()
    reg5.fit(X3_Poly,dfsearchWoOutliers[['conv']])
    scores5 = cross_val_score(reg5, X3_Poly, dfsearchWoOutliers[['conv']])
    #print (reg5.coef_)
    #print (reg5.intercept_)
    #print ("with ATR,CTR3rdDegree",scores5.mean())
    mean_scores.append(scores5.mean())
    
    neigh = KNeighborsRegressor(n_neighbors=10)
    neigh.fit(dfsearchWoOutliers[['CTR','ATR']],dfsearchWoOutliers[['conv']])
    scores6 = cross_val_score(neigh, dfsearchWoOutliers[['CTR','ATR']], dfsearchWoOutliers[['conv']])                                                                   
    #print ("Neighbour",scores6.mean())
    mean_scores.append(scores6.mean())
    
    reg7 = RandomForestRegressor()
    reg7.fit(dfsearchWoOutliers[['CTR']],dfsearchWoOutliers[['conv']])
    scores7 = cross_val_score(reg7, dfsearchWoOutliers[['CTR']], dfsearchWoOutliers[['conv']])
    #print ("with randomforest",scores7.mean())
    mean_scores.append(scores7.mean())
    
    counter_meanscores = list(enumerate(mean_scores, 1))
    counter_list_sorted = sorted(counter_meanscores, key=lambda x:x[1],reverse=True)
    #print (counter_list_sorted[0][1])
    
    if counter_list_sorted[0][1]>=.2:
        if counter_list_sorted[0][0]==1:
            return reg0,counter_list_sorted[0][1],counter_list_sorted[0][0]
        elif counter_list_sorted[0][0]==2:
            return reg1,counter_list_sorted[0][1],counter_list_sorted[0][0]
        elif counter_list_sorted[0][0]==3:
            return reg2,counter_list_sorted[0][1],counter_list_sorted[0][0]
        elif counter_list_sorted[0][0]==4:
            return reg3,counter_list_sorted[0][1],counter_list_sorted[0][0]
        elif counter_list_sorted[0][0]==5:
            return reg4,counter_list_sorted[0][1],counter_list_sorted[0][0]
        elif counter_list_sorted[0][0]==6:
            return reg5,counter_list_sorted[0][1],counter_list_sorted[0][0]
        elif counter_list_sorted[0][0]==7:
            return neigh,counter_list_sorted[0][1],counter_list_sorted[0][0]
        elif counter_list_sorted[0][0]==8:
            return reg7,counter_list_sorted[0][1],counter_list_sorted[0][0]
    else:
        return None

In [366]:
##for single dimension input
def plotregresmodel(model,datax,datay,datainlist):
    plt.scatter(datax,datay)
    plt.plot(datax,model.predict(datainlist),color='blue',linewidth=3)
    plt.show()

In [367]:
##no default model as global data doesn't have any trend 
#def modelordefault(searchstring):

In [368]:
#def predict(ctr,atr,searchstring):

In [369]:
dfsearchWoOutliers = removeoutliers("canadiana")
print (highfreq_searchStrings[2])
model,r2,number = Regressors(dfsearchWoOutliers)
print (model,"R-squarer",r2,"model-num",number)
joblib_file_name = ('trial'+'.joblib')
fileObject = open('./pickle/'+joblib_file_name,'wb')
joblib.dump(model, fileObject)
fileObject.close()
#models[searchstring] = [joblib_file_name,r2,number]
#joblib.dumps(models)
fileObject = open('./pickle/'+joblib_file_name,'rb')
b = joblib.load(fileObject)
print ("open",b)
#pairplotgraphs(dfsearchWoOutliers)
#plotregresmodel(abc[0],dfsearchWoOutliers['CTR'],dfsearchWoOutliers['conv'],dfsearchWoOutliers[['CTR']])

cat food
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False) R-squarer 0.24386816843 model-num 2
open LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)


In [370]:
## for global model
#dfsearchWoOutliers = removeoutliers()
#Regressors(dfsearchWoOutliers)
#pairplotgraphs(dfsearchWoOutliers)

In [ ]:
models={}
for searchstring in highfreq_searchStrings:
    #print (searchstring)
    dfsearchWoOutliers = removeoutliers(searchstring)
    if Regressors(dfsearchWoOutliers)!=None:
        model,r2,number = Regressors(dfsearchWoOutliers)
        name = ''.join(x for x in searchstring if x.isalpha())
        #print (name)
        joblib_file_name = (name+'.joblib')
        #print (joblib_file_name)
        fileObject = open('./pickle/'+joblib_file_name,'wb')
        joblib.dump(model, fileObject)
        fileObject.close()
        models[searchstring] = [joblib_file_name,r2,number]                   
        #pairplotgraphs(dfsearchWoOutliers)
        #print (model,r2,number)
fileObject = open("./pickle/models.joblib",'wb')
joblib.dump(models, fileObject)
fileObject.close()

In [ ]:
#fileObject = open("./pickle/initial-dataframe.joblib",'wb')
#joblib.dump(df2, fileObject)
#fileObject.close()

In [ ]:
#recommendtop10products if string present in model and top 10 rule is that if the predicted value of conversion 
#is higher than existing value then add it to a list with the conversion values and then sort the list based on
#conversion values and show the top 10

def recommendtop10products():
    recommendations={}
    fileObject = open("./pickle/models.joblib",'rb')
    model_dict = joblib.load(fileObject)
    #fileObject = open("./pickle/initial-dataframe.joblib",'rb')
    #initial_dataframe = joblib.load(fileObject)
    #print ("model",model_dict)
    for keys,values in model_dict.items():
        df=df2[(df2['searchTerm'] == keys)]
        df = df.reset_index(drop=True)
        #print (df.loc[0])
        #print (df[['CTR','ATR']])
        print (keys,values)
        fileObject1 = open(('./pickle/'+values[0]),'rb')
        regmodel = joblib.load(fileObject1)
        #print (regmodel)
        if values[2]==1:
            results=regmodel.predict(df[['CTR']])
        elif values[2]==2:
            results = regmodel.predict(df[['ATR']])
        elif values[2]==3:
            results = regmodel.predict(df[['CTR','ATR']])
        elif values[2]==4:
            results = regmodel.predict(df[['CTR','ATR','ATRmCTR']])
        elif values[2]==5:
            poly2 = PolynomialFeatures(degree=2)
            X2_Poly = poly2.fit_transform(df[['CTR','ATR']])
            results = regmodel.predict(X2_Poly)
        elif values[2]==6:
            poly3 = PolynomialFeatures(degree=3)
            X3_Poly = poly3.fit_transform(df[['CTR','ATR']])
            results = regmodel.predict(X3_Poly)
        elif values[2]==7:
            results = regmodel.predict(df[['CTR','ATR']])
        elif values[2]==8:
            results = regmodel.predict(df[['CTR']])
            
        results = results.tolist()
        results = list(enumerate(results,0))
        results = sorted(results, key=lambda x:x[1],reverse=True)
        ## take top 10 results from the sorted enumerated results and use the position to get the products from 
        #dataframe
        productlist=[]
        for i in range(10):
            productlist.append(df.loc[results[i][0]]['product_id'])
        recommendations[keys]=[productlist]
        fileObject1.close()
    fileObject.close()
    return recommendations       

In [ ]:
recommendations = recommendtop10products()
print (recommendations)